In [24]:

#
#
# imports and definitions
#
#
import os
import numpy as np

import time
import sys
import numpy as np
import scipy.stats as st
import networkx as nx
import itertools as it
import random as rd
import os.path
from collections import defaultdict
import time
from sklearn.preprocessing import normalize



def rnd_walk_matrix(A, r, alpha, num_nodes):
    
    n = num_nodes
    M = normalize(A, norm='l1', axis=0)                                 # column wise normalized MArkov matrix
    factor = float((1-alpha)/num_nodes)
#     print(factor)
    E = np.multiply(factor,np.ones([num_nodes,num_nodes]))              # prepare 2nd scaling term
    M2 = np.multiply(alpha,M) + E                                       # mixture of Markov chains
    del M
    del E
    
    U = np.identity(n,dtype=int) 
    H = (1-r)*M2
    H1 = np.subtract(U,H)
    del U
    del M2
    del H    

    W = r*np.linalg.inv(H1)                                             # calculate random-walk matrix
    del H1
    return W

In [28]:
#
#
# defining the input: parameters, seed sets and network
#
#


rlist = [.9] # restart probabiltiy 
alpha = float(1)
r=.9

#nf_literature_proteins = ['4763', '3845',  '4893', '3265', '22882', '5609', '5594', '161742', '5291', '207', '2475']


diseasegenes_dict={} # the dictionary where you keep your diseas genes 
#diseasegenes_dict['NF']=['4763', '3845',  '4893', '3265', '22882', '5609', '5594', '161742', '5291', '207', '2475']
#diseasegenes_dict['ALPS']=['355', '356', '841', '836', '839', '840']
#diseasegenes_dict['HEM1']=['3071', '207', '10006' ,'8936', '60', '253260','2475','79109','64223']
#diseasegenes_dict['WDR1']=['9948', '60', '1072','818', '11034']
#diseasegenes_dict['TTC7A']=['57217', '855252', '5297', '84668', '23209', '3690']

'''
diseasegenes_dict['PTPN2']=['5771']
diseasegenes_dict['FERMT1']=['55612']
diseasegenes_dict['RPIA']=['22934']
diseasegenes_dict['RAG1']=['5896']
diseasegenes_dict['BLM']=['641']


diseasegenes_dict['CTLA4']=['1493']
diseasegenes_dict['AIRE']=['326']
diseasegenes_dict['CFTR']=['1080']
diseasegenes_dict['RPSA']=['3921']
diseasegenes_dict['ADA2']=['51816']
diseasegenes_dict['GATA2']=['2624']
diseasegenes_dict['HTT']=['3064']
diseasegenes_dict['UBE3A']=['7337']
diseasegenes_dict['TYR']=['7299']
diseasegenes_dict['SERPINEA1']=['5265']
'''

diseasegenes_dict['HEM1']=['3071','60','10006','8936','10787','23191','26999','55845','253260','2475','79109','64223','55615','79899','207']

# this will be your network
networkfile= '/Users/Iva/Documents/project/data/monster_clean.txt'
#this will be your dictionary of seed genes (the nodes that you will start the random walk from)

seedgroups=diseasegenes_dict

In [29]:
#
#
# creating the random walk matrix of the network
#
#

G0 = nx.read_edgelist(networkfile)
#G = max(nx.connected_component_subgraphs(G0), key=len)
G = max((G0.subgraph (c) for c in nx.connected_components(G0)), key=len)

if G.number_of_nodes() < G0.number_of_nodes():
    print('Network is NOT connected! Giant component taken.')
else:
    print('Network is connected.')

print('lcc network: #nodes=%s , # edges=%s ' %(G.number_of_nodes(),G.number_of_edges()))

# ###########################################################################
#                                                                           #
#    CALCULATES MATRIX INVERSION FOR GIVEN PARAMETERS                       #
#                                                                           #
print('calculating adjacency matrix')                                                                            #
num_nodes = G.number_of_nodes()                                             #
A = nx.adjacency_matrix(G, sorted(G.nodes()))                               #
                                                                       #
#################################################################           #
#                                                                           #
#    INVERT MARKOV MATRIX & GENERATE RW MATRIX                              #
#                                                                           #
###################                                                         #
t0 = time.time()
W = rnd_walk_matrix(A, r, alpha, num_nodes)                                 #
                                                                        #
print('Inversion done, with r== %s' %(r))                                   #
#################################################################           #
#                                                                           #
#    GENERATE DICT FOR NODE-LABELS TO INTEGERS                              #
#                                                                           #
###################                                                         #
                                                                        #
d_idx_entz = {}                                                             #
cc = 0                                                                      #
for entz in sorted(G.nodes()):                                              #
    d_idx_entz[cc] = entz                                                   #
    cc += 1                                                                 #
d_entz_idx = dict((y,x) for x,y in d_idx_entz.items())                      #
print('rnd walk computing time: %.2f' %float(time.time()-t0))               #
                                                                        #
#############################################################################


Network is NOT connected! Giant component taken.
lcc network: #nodes=18818 , # edges=483257 
calculating adjacency matrix
Inversion done, with r== 0.9
rnd walk computing time: 416.73


In [30]:

#
#
# going through each seed node and making the visit probability dicitonary
#
#

visit_prob_dictionaries={}  ##name of dictionaries inside is gene names
for thegroup, l_ent in seedgroups.items(): 
    seeds= thegroup
    if (set(l_ent).issubset(set(G.nodes()))):
        print('All genes in network.')
        l_ent2cont = l_ent
    else:
        l_ent2cont = []
        for g in l_ent:
            if g in G:
                l_ent2cont.append(g)
            else:
                print('Genes %s is not part of the network and will be neglected!' %g)
    print('number of genes in set: %s' %len(l_ent))
    print('number of set genes on network: %s' %len(l_ent2cont))

    # make list of indices
    nodeset = []
    for s_ent in l_ent2cont:
        nodeset.append(d_entz_idx[s_ent])

    normprob = len(nodeset)
    print('Size of initial set: %s seed genes' %normprob)
    p0 = np.zeros(G.number_of_nodes())
    # generate start vector
    for n in range(len(p0)):
        if n in nodeset:
            p0[n] = 1.
    print('check for sum of initial p-vec')
    print(np.sum(p0))
    pinf =  np.array(W.dot(p0))

    # DICT WITH NODE ID AND PVIS
    d_n_p = {}
    i = 0
    for x in pinf[0]:
        #     print(i,x)
        d_n_p[d_idx_entz[i]] = x/normprob
        i += 1

    # write file

    pcum = 0
    i=0
    sorted_d_n_p= sorted(d_n_p.items(), key = lambda x: x[1], reverse = True)
    visit_prob_dictionaries[thegroup]=sorted_d_n_p
    

All genes in network.
number of genes in set: 15
number of set genes on network: 15
Size of initial set: 15 seed genes
check for sum of initial p-vec
15.0


In [31]:
len (visit_prob_dictionaries)
print (visit_prob_dictionaries.keys())

#nf_rw = visit_prob_dictionaries["NF"]
#alps_rw = visit_prob_dictionaries["ALPS"]
#hem1_rw = visit_prob_dictionaries["HEM1"]
#wdr1_rw = visit_prob_dictionaries["WDR1"]
#ttc7a_rw = visit_prob_dictionaries["TTC7A"]
#rictor_rw = visit_prob_dictionaries['RICTOR']
'''
ptpn2_rw = visit_prob_dictionaries['PTPN2']
fermit1_rw = visit_prob_dictionaries['FERMT1']
rpia_rw=visit_prob_dictionaries['RPIA']
rag1_rw=visit_prob_dictionaries['RAG1']
blm_rw=visit_prob_dictionaries['BLM']




ctla4_rw = visit_prob_dictionaries['CTLA4']
aire_rw = visit_prob_dictionaries['AIRE']
cftr_rw = visit_prob_dictionaries['CFTR']
rpsa_rw = visit_prob_dictionaries['RPSA']
ada2_rw = visit_prob_dictionaries['ADA2']
gata2_rw = visit_prob_dictionaries['GATA2']
htt_rw = visit_prob_dictionaries['HTT']
ube3a_rw = visit_prob_dictionaries['UBE3A']
tyr_rw = visit_prob_dictionaries['TYR']
serpinea1_rw = visit_prob_dictionaries['SERPINEA1']

'''

hem1 = visit_prob_dictionaries['HEM1']

dict_keys(['HEM1'])


In [32]:
   
f1 = open ("hem1_literature_all_rw.txt", "w")
for j in hem1:   
    f1.write(j[0] + "\t" + str(j[1]) + "\n")
f1.close()


'''
f2 = open ("aire_rw.txt", "w")
for j in aire_rw:   
    f2.write(j[0] + "\t" + str(j[1]) + "\n")
f2.close()

f3 = open ("cftr_rw.txt", "w")
for j in cftr_rw:   
    f3.write(j[0] + "\t" + str(j[1]) + "\n")
f3.close()

f4 = open ("rpsa_rw.txt", "w")
for j in rpsa_rw:   
    f4.write(j[0] + "\t" + str(j[1]) + "\n")
f4.close()

f5 = open ("ada2_rw.txt", "w")
for j in ada2_rw:   
    f5.write(j[0] + "\t" + str(j[1]) + "\n")
f5.close()

f6 = open ("gata2_rw.txt", "w")
for j in gata2_rw:   
    f6.write(j[0] + "\t" + str(j[1]) + "\n")
f6.close()

f7 = open ("htt_rw.txt", "w")
for j in htt_rw:   
    f7.write(j[0] + "\t" + str(j[1]) + "\n")
f7.close()

f8 = open ("ube3a_rw.txt", "w")
for j in ube3a_rw:   
    f8.write(j[0] + "\t" + str(j[1]) + "\n")
f8.close()

f9 = open ("tyr_rw.txt", "w")
for j in tyr_rw:   
    f9.write(j[0] + "\t" + str(j[1]) + "\n")
f9.close()

f10 = open ("serpinea1_rw.txt", "w")
for j in serpinea1_rw:   
    f10.write(j[0] + "\t" + str(j[1]) + "\n")
f10.close()





f1 = open ("rictor_rw.txt", "w")
for j in rictor_rw:   
    f1.write(j[0] + "\t" + str(j[1]) + "\n")
f1.close()

''' 

'\nf2 = open ("aire_rw.txt", "w")\nfor j in aire_rw:   \n    f2.write(j[0] + "\t" + str(j[1]) + "\n")\nf2.close()\n\nf3 = open ("cftr_rw.txt", "w")\nfor j in cftr_rw:   \n    f3.write(j[0] + "\t" + str(j[1]) + "\n")\nf3.close()\n\nf4 = open ("rpsa_rw.txt", "w")\nfor j in rpsa_rw:   \n    f4.write(j[0] + "\t" + str(j[1]) + "\n")\nf4.close()\n\nf5 = open ("ada2_rw.txt", "w")\nfor j in ada2_rw:   \n    f5.write(j[0] + "\t" + str(j[1]) + "\n")\nf5.close()\n\nf6 = open ("gata2_rw.txt", "w")\nfor j in gata2_rw:   \n    f6.write(j[0] + "\t" + str(j[1]) + "\n")\nf6.close()\n\nf7 = open ("htt_rw.txt", "w")\nfor j in htt_rw:   \n    f7.write(j[0] + "\t" + str(j[1]) + "\n")\nf7.close()\n\nf8 = open ("ube3a_rw.txt", "w")\nfor j in ube3a_rw:   \n    f8.write(j[0] + "\t" + str(j[1]) + "\n")\nf8.close()\n\nf9 = open ("tyr_rw.txt", "w")\nfor j in tyr_rw:   \n    f9.write(j[0] + "\t" + str(j[1]) + "\n")\nf9.close()\n\nf10 = open ("serpinea1_rw.txt", "w")\nfor j in serpinea1_rw:   \n    f10.write(j[0] +